In [1]:
import cv2
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Specify the model to be used
COCO and MPI are body pose estimation model. COCO has 18 points and MPI has 15 points as output.

HAND is hand keypoints estimation model. It has 22 points as output

Ensure that the model files are available in the folders.

In [2]:
MODE = "MPI"

# if MODE is "COCO":
#     protoFile = "pose/coco/pose_deploy_linevec.prototxt"
#     weightsFile = "pose/coco/pose_iter_440000.caffemodel"
#     nPoints = 18
#     POSE_PAIRS = [ [1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]

# elif MODE is "MPI" :
protoFile = "pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose/mpi/pose_iter_160000.caffemodel"
nPoints = 15
POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] ]
    

#### Let us load an image with multiple people and check what the model sees

In [3]:
# image1 = cv2.imread("multiple.jpeg")
# frameWidth = image1.shape[1]
# frameHeight = image1.shape[0]
# threshold = 0.1

#### Load the network and pass the image through the network

In [4]:
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

inWidth = 368
inHeight = 368
# inpBlob = cv2.dnn.blobFromImage(image1, 1.0 / 255, (inWidth, inHeight),
#                           (0, 0, 0), swapRB=False, crop=False)

# net.setInput(inpBlob)
# output = net.forward()
# H = output.shape[2]
# W = output.shape[3]
# print(output.shape)

#### Slice a probability map from the output for a specific keypoint and plot the heatmap ( after resizing ) on the image itself

In [5]:
# i = 5
# probMap = output[0, i, :, :]
# probMap = cv2.resize(probMap, (image1.shape[1], image1.shape[0]))
# plt.figure(figsize=[14,10])
# plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
# plt.imshow(probMap, alpha=0.6)
# plt.colorbar()
# plt.axis("off")

#### Similarly plot the affinity map on the image

In [6]:
# i = 24
# probMap = output[0, i, :, :]
# probMap = cv2.resize(probMap, (image1.shape[1], image1.shape[0]))
# plt.figure(figsize=[14,10])
# plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
# plt.imshow(probMap, alpha=0.6)
# plt.colorbar()
# plt.axis("off")

#### Next, we find the keypoints for a image with only single person

In [23]:
counter = -1
df_is_empty = True
for filename in os.listdir('.'):
    if filename.endswith(".jpg"): 
#         try:
        counter += 1
        frame = cv2.imread(filename)
        frameCopy = np.copy(frame)
        frameWidth = frame.shape[1]
        frameHeight = frame.shape[0]
        threshold = 0.1
        is_None = False
        
#         point_df = []
        inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                                  (0, 0, 0), swapRB=False, crop=False)

        net.setInput(inpBlob)

        output = net.forward()
        H = output.shape[2]
        W = output.shape[3]

        points = []

        for i in range(nPoints):
            # confidence map of corresponding body's part.
            probMap = output[0, i, :, :]

            # Find global maxima of the probMap.
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

            # Scale the point to fit on the original image
            x = (frameWidth * point[0]) / W
            y = (frameHeight * point[1]) / H

            if prob > threshold : 
                cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
                cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
                cv2.circle(frame, (int(x), int(y)), 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

                # Add the point to the list if the probability is greater than the threshold
                points.append((int(x), int(y)))
            else :
                points.append(None)
                is_None = True
              
        if is_None:
            continue
        flat_point = [e for l in points for e in l]
#         print(flat_point)
        flat_array = np.array([e for l in points for e in l])/400
        point_dict = {i:flat_array[i] for i in np.arange(len(flat_array))}

        if filename.find('right') >= 0:
            point_dict['label'] = 1
        else:
            point_dict['label'] = 0
#         if len(point_df) == 0:
#             point_df = pd.DataFrame(point_dict, index=[0])
#         else:
#             point_df = point_df.append(point_dict, ignore_index=True)

#         flat_array = flat_array / flat_array.max()
#         print(f"{flat_array} - {flat_array.max()}\n\n")
        # Draw Skeleton
        
        if df_is_empty:
            point_df = pd.DataFrame(point_dict, index=[0])
            df_is_empty = False
        else:
            point_df = point_df.append(point_dict, ignore_index=True)
            
        for pair in POSE_PAIRS:
            partA = pair[0]
            partB = pair[1]

            if points[partA] and points[partB]:
                cv2.line(frame, points[partA], points[partB], (0, 255, 255), 3)
        ##points
        
#         plt.figure(figsize=[10,10])
#         plt.imshow(cv2.cvtColor(frameCopy, cv2.COLOR_BGR2RGB))
        print(counter)
    print(filename)
        
#         except:
#             print(filename)
#             continue
            

        ##lines
#         plt.figure(figsize=[10,10])
#         plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

range_of_motion.ipynb
1
wrong_resized_image_2471.jpg
pose
2
wrong_resized_image_2470.jpg
Roee_5.ipynb
3
wrong_resized_image_2508.jpg
5
wrong_resized_image_2510.jpg
6
wrong_resized_image_2518.jpg
rotate.py
.ipynb_checkpoints
8
wrong_resized_image_2509.jpg
10
wrong_resized_image_2480.jpg
get_images_from_video.ipynb
11
wrong_resized_image_2478.jpg
IMG-1844.TRIM.MOV
12
wrong_resized_image_2521.jpg
resize.py
13
wrong_resized_image_2520.jpg
14
wrong_resized_image_2477.jpg
16
wrong_resized_image_2519.jpg


In [21]:
point_df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,label
0,0.3250,0.1725,0.3675,0.3025,0.4125,0.3250,0.3900,0.4125,0.3250,0.4550,...,0.8475,0.5200,0.5650,0.3475,0.6075,0.4325,0.8475,0.4775,0.4550,0.0
1,0.3250,0.1500,0.3675,0.2825,0.4125,0.3025,0.3675,0.4125,0.3250,0.4325,...,0.8250,0.5200,0.5425,0.3675,0.6300,0.4125,0.8475,0.4550,0.4325,0.0
2,0.3250,0.1075,0.3675,0.2375,0.4125,0.2600,0.3675,0.3675,0.3025,0.4125,...,0.8025,0.5200,0.5000,0.3675,0.6075,0.4325,0.8250,0.4775,0.3900,0.0
3,0.3250,0.1500,0.3675,0.2825,0.4125,0.3025,0.3675,0.4125,0.3475,0.4550,...,0.8250,0.5425,0.5425,0.3675,0.6075,0.4125,0.8475,0.4775,0.4550,0.0
4,0.3250,0.1725,0.3675,0.3025,0.4125,0.3250,0.3675,0.4125,0.3250,0.3900,...,0.8475,0.5200,0.5425,0.3475,0.6075,0.4325,0.8250,0.4775,0.4550,0.0
5,0.3250,0.1300,0.3675,0.2600,0.4125,0.2825,0.3675,0.3900,0.2825,0.4125,...,0.8250,0.5425,0.5200,0.3475,0.6075,0.4325,0.8675,0.4775,0.4125,0.0
6,0.3250,0.1500,0.3475,0.2600,0.3900,0.3025,0.3250,0.4325,0.2825,0.4325,...,0.8475,0.5200,0.5000,0.3475,0.6075,0.4550,0.8675,0.4550,0.4125,0.0
7,0.3250,0.1950,0.3475,0.3250,0.3900,0.3475,0.3475,0.4125,0.2825,0.3675,...,0.8475,0.5425,0.5425,0.3475,0.6075,0.4325,0.8475,0.4550,0.4550,0.0
8,0.3025,0.1300,0.3675,0.2375,0.4125,0.2600,0.3675,0.4125,0.3025,0.4125,...,0.8025,0.5425,0.4775,0.3900,0.6075,0.4325,0.8475,0.4550,0.3675,0.0
9,0.3250,0.1300,0.3675,0.2600,0.4125,0.2825,0.3475,0.4325,0.3025,0.4325,...,0.8250,0.5425,0.5000,0.3900,0.6300,0.4325,0.8250,0.4775,0.4125,0.0


#### Pass it through the network

In [ ]:
# inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
#                           (0, 0, 0), swapRB=False, crop=False)

# net.setInput(inpBlob)

# output = net.forward()
# H = output.shape[2]
# W = output.shape[3]

#### gather the points and plot the keypoints and the skeleton figure

In [ ]:
# Empty list to store the detected keypoints
# points = []

# for i in range(nPoints):
#     # confidence map of corresponding body's part.
#     probMap = output[0, i, :, :]

#     # Find global maxima of the probMap.
#     minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
    
#     # Scale the point to fit on the original image
#     x = (frameWidth * point[0]) / W
#     y = (frameHeight * point[1]) / H

#     if prob > threshold : 
#         cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
#         cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
#         cv2.circle(frame, (int(x), int(y)), 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

#         # Add the point to the list if the probability is greater than the threshold
#         points.append((int(x), int(y)))
#     else :
#         points.append(None)

# # Draw Skeleton
# for pair in POSE_PAIRS:
#     partA = pair[0]
#     partB = pair[1]

#     if points[partA] and points[partB]:
#         cv2.line(frame, points[partA], points[partB], (0, 255, 255), 3)

# plt.figure(figsize=[10,10])
# plt.imshow(cv2.cvtColor(frameCopy, cv2.COLOR_BGR2RGB))
# plt.figure(figsize=[10,10])
# plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))